In [2]:
import requests
import lxml.html as html
import pandas as pd
import re
from selenium import webdriver
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
from open_sheet import open_sheet

import progressbar
import time

## Pao de acucar

In [2]:
#Modificar
web = 'https://www.paodeacucar.com/busca?w=ricota&qt=12&p=1&gt=grid'

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
path = './chromedriver'
web_driver = webdriver.Chrome(path, options=options)
web_driver.get(web)

In [6]:
#click a entiendo
entendi_button = web_driver.find_element_by_xpath('//*[@class="buttonstyles__ButtonStyle-sc-1mux0mx-0 alert-cookiestyles__Button-ubrksr-3 bjGqGj"]')
entendi_button.click()

In [7]:
seccion = web_driver.find_elements_by_xpath('//div[@class="breadcrumbstyles__Section-nzvsip-3 eWwJcz"]')[-1].text
total = int(web_driver.find_element_by_xpath('//div[@class="MuiGrid-root gridstyles-sc-6scn59-0 jSNVZP MuiGrid-item"]').text.split(' ')[0])
print('Total de productos encontrados: ' + str(total))

Total de productos encontrados: 18


In [8]:
prod_pao = []
for i in progressbar.progressbar(range(total)):
    prod = web_driver.find_elements_by_xpath('//div[@class="MuiGrid-root gridstyles-sc-6scn59-0 cZKfLv MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-sm-6 MuiGrid-grid-md-4 MuiGrid-grid-lg-3 MuiGrid-grid-xl-3"]')[i]
    web_driver.execute_script("arguments[0].scrollIntoView(false);", prod)
    time.sleep(0.6)
    name = web_driver.find_elements_by_xpath('//text[@class="product-cardstyles__Text-sc-1uwpde0-6 jVbxeZ"]/a')[i].text
    url = web_driver.find_elements_by_xpath('//text[@class="product-cardstyles__Text-sc-1uwpde0-6 jVbxeZ"]/a')[i].get_attribute('href')
    photo_link = web_driver.find_elements_by_xpath('//div[@class="placeholderstyles__PlaceholderContainer-ix9p6w-0 product-cardstyles__ImageDiv-sc-1uwpde0-2 lgyKfL"]/img')[i].get_attribute('src')

    prod_pao.append({'name': name,
                     'url':url,
                     'seccion':seccion,
                     'photo_link':photo_link})
web_driver.quit()

100% (18 of 18) |########################| Elapsed Time: 0:00:12 Time:  0:00:12


In [9]:
df_prod_pao = pd.DataFrame(prod_pao)
df_prod_pao

,name,url,seccion,photo_link
0,Ricota Fresca QUATÁ 400g,https://www.paodeacucar.com/produto/6019/ricot...,ricota,https://static.paodeacucar.com/img/uploads/1/7...
1,Creme de Ricota Light Polenghi 150g,https://www.paodeacucar.com/produto/297278/cre...,ricota,https://static.paodeacucar.com/img/uploads/1/2...
2,Creme de Queijo Ricota Light Tirolez Pote 200g,https://www.paodeacucar.com/produto/451667/cre...,ricota,https://static.paodeacucar.com/img/uploads/1/6...
3,Creme de Ricota Zero Lactose TIROLEZ 150g,https://www.paodeacucar.com/produto/397579/cre...,ricota,https://static.paodeacucar.com/img/uploads/1/5...
4,Creme de Ricota DANUBIO 200g,https://www.paodeacucar.com/produto/336737/cre...,ricota,https://static.paodeacucar.com/img/uploads/1/5...
5,Torta de Ricota com Espinafre QUALITÁ 500g,https://www.paodeacucar.com/produto/423453/tor...,ricota,https://static.paodeacucar.com/img/uploads/1/3...
6,Torta com Massa de Iogurte Recheio Espinafre c...,https://www.paodeacucar.com/produto/462495/tor...,ricota,https://static.paodeacucar.com/img/uploads/1/9...
7,Lasanha Integral de Frango com Molho de Ricota...,https://www.paodeacucar.com/produto/393444/las...,ricota,https://static.paodeacucar.com/img/uploads/1/4...
8,Creme De Ricota QUATÁ Pote 150g,https://www.paodeacucar.com/produto/223328,ricota,http://www.paodeacucar.com/img/uploads/1/832/5...
9,Creme De Ricota Com Presunto QUATÁ Pote 150g,https://www.paodeacucar.com/produto/481484,ricota,http://www.paodeacucar.com/img/uploads/1/787/1...


In [11]:
for pr in progressbar.progressbar(range(len(df_prod_pao))):
    
    #Ingreso a la web
    web = df_prod_pao.loc[pr,'url']
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    path = './chromedriver'
    web_prord = webdriver.Chrome(path, options=options)
    web_prord.get(web)
    
    #click a entiendo
    entendi_button = web_prord.find_element_by_xpath('//*[@class="buttonstyles__ButtonStyle-sc-1mux0mx-0 alert-cookiestyles__Button-ubrksr-3 bjGqGj"]')
    entendi_button.click()
    
    try:
        df_prod_pao.loc[pr,'Descripcion_producto'] = web_prord.find_element_by_xpath('//div[@class="MuiGrid-root gridstyles-sc-6scn59-0 laucrw"]').text
    except:
         df_prod_pao.loc[pr,'Descripcion_producto'] = ''
            
    web_prord.quit()

100% (18 of 18) |########################| Elapsed Time: 0:01:38 Time:  0:01:38


In [13]:
scope_gdoc = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
cre = ServiceAccountCredentials.from_json_keyfile_name('', scope_gdoc)
gc = gspread.authorize(cre)

In [15]:
#Modificar
#Planilla a donde se guarda el resultado
Planilla = ''
#Nombre del dominio + pao deacucar
sheet = ''

In [16]:
d2g.upload(df_prod_pao.reset_index(drop=True).fillna(''),
           Planilla,sheet,credentials=cre, row_names=False)

<Worksheet 'Ricotta - poa de acucar' id:919218669>